In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

#### Write your own GNN module
* Sometimes, you model goes **beyongd simply stacking existing GNN modules**. 
* For example, you can invent a **new way of aggregating neighbor information** by **considering node importance** or edge weights.
* In the section, you will learn:
   1. Understand DGL's message passing APIs.
   2. Implement GraphSAGE convolution module by your own. 
* 这一节是学会搭建自己的GNN模块，自己进行细节设计，比如可以设计一个**新的Aggregation operation**等等。这一节会学习DGL里面内置的message passing APIs，即**通过消息传递实现更新**的操作。

* 目前GNN中主要操作其实比较简单，本质上就是消息传递，然后聚合更新，在聚合更新上之前出了很多paper
* 下面是一个NIPS'17的paper实现，dgl里面也有对应API，dgl.nn.SAGEConv

In [ ]:
import dgl.function as fn

class SAGEConv(nn.Module):
    '''
    Parameters
    ----------
    in_feat(int): input feature size.
    out_feat(int): output feature size.
    '''
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodel for projecting the input and neighbor feature to the output
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h):
        '''
        Parameters
        ----------
        g: Graph, the input graph
        h: Tensor, the input feature
        '''
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)
    